In [1]:
import sys
sys.path.insert(0, "../scripts")
import helpers as h
import window_detect as w

import pandas as pd
import plotly.graph_objects as go
import math


In [2]:
ea = w.Window_Detect("A")
ea.main()

next is null, but predict close 15


TypeError: ufunc 'bitwise_xor' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [3]:
clean_deriv = ea.guess
clean_deriv.loc[:, "adjust_deriv"] = clean_deriv["deriv"] - 0.5
ws_df = clean_deriv.reset_index(drop=True)
ws_df["shift_adjust_deriv"] = ws_df["adjust_deriv"].shift(-1)
ws_df

,index,deriv,adjust_deriv,shift_adjust_deriv
0,2022-07-20 07:30:00,0.014985,-0.485015,0.490955
1,2022-07-20 12:15:00,0.990955,0.490955,-0.323836
2,2022-07-20 22:30:00,0.176164,-0.323836,0.456238
3,2022-07-21 13:00:00,0.956238,0.456238,-0.428882
4,2022-07-21 14:45:00,0.071118,-0.428882,0.457060
5,2022-07-21 17:15:00,0.957060,0.457060,-0.323836
6,2022-07-21 22:30:00,0.176164,-0.323836,0.446933
7,2022-07-22 07:30:00,0.946933,0.446933,-0.500000
8,2022-07-22 11:00:00,0.000000,-0.500000,0.442301
9,2022-07-22 15:45:00,0.942301,0.442301,-0.415717


In [4]:
def check_winstate(row):
    curr = row["adjust_deriv"]
    next = row["shift_adjust_deriv"]
    # print(row["adjust_deriv"], row["shift_adjust_deriv"],)
    if curr < 0 and next > 0:
        return 1
    elif curr > 0 and next < 0:
        return 0
    elif curr > 0 and math.isnan(next) :
        print("hello!", row.name )
        return 0
    elif curr < 0 and math.isnan(next) :
        print("hello 2!", row.name )
        return 1
    else:
        return None

In [5]:
ws_df["state"] = ws_df.apply(check_winstate, axis=1 )
ws_df

hello! 11


,index,deriv,adjust_deriv,shift_adjust_deriv,state
0,2022-07-20 07:30:00,0.014985,-0.485015,0.490955,1
1,2022-07-20 12:15:00,0.990955,0.490955,-0.323836,0
2,2022-07-20 22:30:00,0.176164,-0.323836,0.456238,1
3,2022-07-21 13:00:00,0.956238,0.456238,-0.428882,0
4,2022-07-21 14:45:00,0.071118,-0.428882,0.457060,1
5,2022-07-21 17:15:00,0.957060,0.457060,-0.323836,0
6,2022-07-21 22:30:00,0.176164,-0.323836,0.446933,1
7,2022-07-22 07:30:00,0.946933,0.446933,-0.500000,0
8,2022-07-22 11:00:00,0.000000,-0.500000,0.442301,1
9,2022-07-22 15:45:00,0.942301,0.442301,-0.415717,0


In [6]:
ws_df["state"]


0     1
1     0
2     1
3     0
4     1
5     0
6     1
7     0
8     1
9     0
10    1
11    0
Name: state, dtype: int64

In [7]:
ws_df["state"][0]

1

In [8]:
# consider the initial value, since only changes are detected, the initial value is the opposite of what is the first item in the ws_df 
init_guess = ws_df["state"][0]
starting_val = init_guess ^ 1

ws_df.loc[-1] =  [ea.varied_room["DateTime"][0]]  +  [float("nan")]*3 + [starting_val]
ws_df.index = ws_df.index + 1
ws_df = ws_df.sort_index()
ws_df

,index,deriv,adjust_deriv,shift_adjust_deriv,state
0,2022-07-20 07:15:00,NaN,NaN,NaN,0
1,2022-07-20 07:30:00,0.014985,-0.485015,0.490955,1
2,2022-07-20 12:15:00,0.990955,0.490955,-0.323836,0
3,2022-07-20 22:30:00,0.176164,-0.323836,0.456238,1
4,2022-07-21 13:00:00,0.956238,0.456238,-0.428882,0
5,2022-07-21 14:45:00,0.071118,-0.428882,0.457060,1
6,2022-07-21 17:15:00,0.957060,0.457060,-0.323836,0
7,2022-07-21 22:30:00,0.176164,-0.323836,0.446933,1
8,2022-07-22 07:30:00,0.946933,0.446933,-0.500000,0
9,2022-07-22 11:00:00,0.000000,-0.500000,0.442301,1


In [9]:
ws_df2 = ws_df.copy()
ws_df2.set_index(ws_df2["index"].values, drop=False, inplace=True)
check = ws_df2.resample("15T").ffill()["state"]


In [10]:
fig = ea.main()
fig.add_trace(go.Scatter(
    x = check.index,
    y=check,
    mode='lines',
    name="Guess as Time Series", 
    opacity=0.5,
    line=dict( width=4, dash='dot')
))

next is null, but predict close 11


## old method

In [11]:
null_state = pd.DataFrame([1] * len(ea.varied_room.index), ea.varied_room.index, columns=["guess_state"])
null_state.head()

,guess_state
2022-07-20 07:15:00,1
2022-07-20 07:30:00,1
2022-07-20 07:45:00,1
2022-07-20 08:00:00,1
2022-07-20 08:15:00,1


In [12]:
ea.varied_room

,DateTime,Temp C,RH %,Room,Ambient Temp,Ambient RH,Window Open
2022-07-20 07:15:00,2022-07-20 07:15:00,23.4386,52.936100,1.0,14.333333,30.555556,0.0
2022-07-20 07:30:00,2022-07-20 07:30:00,23.4178,52.758200,1.0,14.333333,30.555556,0.0
2022-07-20 07:45:00,2022-07-20 07:45:00,23.4154,52.931800,1.0,14.333333,30.555556,1.0
2022-07-20 08:00:00,2022-07-20 08:00:00,23.5538,52.837900,1.0,14.666667,30.555556,1.0
2022-07-20 08:15:00,2022-07-20 08:15:00,23.6650,52.722367,1.0,14.666667,30.555556,1.0
...,...,...,...,...,...,...,...
2022-07-24 06:15:00,2022-07-24 06:15:00,23.3738,52.782333,1.0,14.611111,30.555556,0.0
2022-07-24 06:30:00,2022-07-24 06:30:00,23.3722,52.877567,1.0,14.611111,30.555556,0.0
2022-07-24 06:45:00,2022-07-24 06:45:00,23.3578,53.006400,1.0,14.611111,30.555556,0.0
2022-07-24 07:00:00,2022-07-24 07:00:00,23.3578,53.126733,1.0,14.722222,30.555556,0.0


In [13]:
ea.varied_room["DateTime"].isin(ws_df["index"])

2022-07-20 07:15:00     True
2022-07-20 07:30:00     True
2022-07-20 07:45:00    False
2022-07-20 08:00:00    False
2022-07-20 08:15:00    False
                       ...  
2022-07-24 06:15:00    False
2022-07-24 06:30:00    False
2022-07-24 06:45:00    False
2022-07-24 07:00:00    False
2022-07-24 07:15:00    False
Name: DateTime, Length: 385, dtype: bool